In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = SyntheticDataset(device=device)

In [ ]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_kde_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_kde_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_kde_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_kde_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

In [7]:
model = logReg(num_features=dataset.XZ_train.shape[1])
trainer_fb_fair(
    model,
    dataset,
    optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4),
    device,
    100,
    128,
    False,
    'SGD',
    20,
)

Training: 100%|██████████| 100/100 [00:10<00:00,  9.56epochs/s]


namespace(train_acc_hist=[0.48171875,
                          0.413359375,
                          0.3690625,
                          0.343359375,
                          0.32296875,
                          0.308203125,
                          0.29703125,
                          0.28875,
                          0.28171875,
                          0.27515625,
                          0.269765625,
                          0.263828125,
                          0.25921875,
                          0.254609375,
                          0.25125,
                          0.247578125,
                          0.24515625,
                          0.241796875,
                          0.23890625,
                          0.23703125,
                          0.234765625,
                          0.233515625,
                          0.232109375,
                          0.23015625,
                          0.228203125,
                          0.22609375,
       

In [ ]:
def lr__model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_kde_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_kde_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_kde_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_kde_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))